このチュートリアルでは，[非線形最適制御入門](https://www.coronasha.co.jp/np/isbn/9784339033182/)のアルゴリズム8.3の非線形モデル予測制御を用いて，[続・ドローン産業応用の全て](https://www.ohmsha.co.jp/book/9784274225802/)の2.1節の静的障害物回避タスクを再現します．2冊とも読み終えていることを前提とし，ライブラリを使う手順のみ書きます．\
まず，(2.1.4)~(2.1.6)より，状態方程式(2.1.1)は以下のように表せます．
$$ \dot{\bm{x}} = \bm{f}(\bm{x}, \bm{u}) = \bm{A x} + \bm{B u} $$
$$
\bm{A} =
\left(
\begin{array}{ccc:cccccc}
    & & & & & & & & \\
    & & & & & & & & \\
    & & & & & & & & \\
    & \large{\bm{O_{6 \times 3}}} & & & & \large{\bm{E}_6} & & & \\
    & & & & & & & & \\
    & & & & & & & & \\
    & & & & & & & & \\ \hdashline
    & & & \alpha_{1x} & & & \alpha_{2x} & & \\
    & \large{\bm{O_{3 \times 3}}} & & & \alpha_{1y} & & & \alpha_{2y} & \\
    & & & & & \alpha_{1z} & & & \alpha_{2z}
\end{array}
\right)
\in \mathbb{R}^{9 \times 9}
$$
$$
\bm{B} =
\begin{pmatrix}
    & & \\
    & & \\
    & & \\
    & \large{\bm{O_{6 \times 3}}} & \\
    & & \\
    & & \\
    & & \\ \hdashline
    \beta_{x} & & \\
    & \beta_{y} & \\
    & & \beta_{z} 
\end{pmatrix}
\in \mathbb{R}^{9 \times 3}
$$
状態$\bm{x} = (p_x \ p_y \ p_z \ v_x \ v_y \ v_z \ a_x \ a_y \ a_z)^\mathsf{T} $は何らかの手法で観測もしくは推定されているとします．\
次に評価関数についてです．(2.1.7)を参考に，バリア関数も含めステージコスト$L$を改めて以下のように設定します．
$$ L = \frac{1}{2}||\bm{x} - \bm{x}_{\mathrm{des}}||_{\bm{Q}}^2 + \frac{1}{2}||\bm{u}||_{\bm{R}}^2 + r_\mathrm{u} P_\mathrm{u}(\bm{u}) + \frac{1}{r_\mathrm{x}} B(\bm{x}) $$
$\bm{Q}, \bm{R}$は正定値対称行列です．\
また，終端コスト$\Phi$を以下のように設定します．
$$ \Phi = \frac{1}{2}||x - x_{\mathrm{des}}||_{\bm{Q}}^2 $$
重みはステージコストと同じものを用いています．もちろん変えることも可能です．\
この例では等式制約は考えないため，ハミルトン関数$H$は以下のようになります．
$$ H = L + \bm{\lambda}^\mathsf{T} \bm{f} $$
`NmpcCgmres`を使うためには，以下の3つを事前に解析的に計算しておく必要があります．
$$ \biggl(\frac{\partial \Phi}{\partial \bm{x}}\biggl)^\mathsf{T}, \ \biggl(\frac{\partial H}{\partial \bm{x}}\biggl)^\mathsf{T}, \ \biggl(\frac{\partial H}{\partial \bm{u}}\biggl)^\mathsf{T} $$
転置をとっているのは，実装時に縦ベクトルとしたほうが都合が良いためです．それぞれ計算すると以下のようになります．
$$ \biggl(\frac{\partial \Phi}{\partial \bm{x}}\biggl)^\mathsf{T} = \bm{Q}(\bm{x} - \bm{x}_{\mathrm{des}}) $$
$$ \biggl(\frac{\partial H}{\partial \bm{x}}\biggl)^\mathsf{T} = \bm{Q}(\bm{x} - \bm{x}_{\mathrm{des}}) + \frac{1}{r_\mathrm{x}} \biggl(\frac{\partial B}{\partial \bm{x}}\biggl)^\mathsf{T} + \bm{A}^\mathsf{T} \bm{\lambda} $$
$$ \biggl(\frac{\partial H}{\partial \bm{u}}\biggl)^\mathsf{T} = \bm{R} \bm{u} + r_\mathrm{u} \biggl(\frac{\partial P_\mathrm{u}}{\partial \bm{u}}\biggl)^\mathsf{T} + \bm{B}^\mathsf{T} \bm{\lambda} $$
ここで，$\partial B / \partial \bm{x}, \partial P_\mathrm{u} / \partial \bm{u}$は以下のように計算できます．
$$ \frac{\partial B}{\partial \bm{x}} = \frac{4}{(r^2 - ((p_x - x_{\mathrm{obs}})^2 + (p_y - y_{\mathrm{obs}})^2))^3} \times ((p_x - x_{\mathrm{obs}}) \ (p_y - y_{\mathrm{obs}}) \ 0 \ 0 \ 0 \ 0 \ 0 \ 0 \ 0) $$
$$ \frac{\partial P_\mathrm{u}}{\partial \bm{u}} = 4 \ \mathrm{max}(0, u_{\phi}^2 + u_{\theta}^2 - \alpha^2) \times (u_{\phi} \ u_{\theta} \ 0) $$
事前準備はできたので，これらの結果を元に制御対象のモデルを実装します．基底クラス`BaseModel`の子クラスとして定義し，クラス定数と仮想関数をオーバーライドすればよいです．

In [ ]:
import numpy as np
from numpy.typing import NDArray

from nmpc_cgmres import BaseModel


class DroneStaticObstacleModel(BaseModel):

    X_DIM = 9
    U_DIM = 3
    C_DIM = 0

    MAX_ANGLE = np.deg2rad(30.)
    MAX_THRUST = 10.

    ALPHA_1X = 0.
    ALPHA_1Y = 0.
    ALPHA_1Z = 0.
    ALPHA_2X = 0.
    ALPHA_2Y = 0.
    ALPHA_2Z = 0.
    BETA_X = 1.
    BETA_Y = 1.
    BETA_Z = 1.

    def __init__(
        self,
        pole_pos: NDArray,
        pole_radius: float,
        vel_weights: NDArray,
        input_weight: float,
        rx: float,
        ru: float,

    ) -> None:
        assert pole_pos.shape == (2,)
        assert pole_radius > 0.
        assert vel_weights.shape == (3,) and np.all(vel_weights > 0.)
        assert input_weight > 0.  # コスト関数の凸性を保証するために正則化項は必須
        assert rx > 0.
        assert ru > 0.

        super().__init__()

        self._rx = rx
        self._ru = ru

        # 柱の位置と半径は事前に正確にわかっているとする
        self._pole_x, self._pole_y = pole_pos
        self._pole_radius = pole_radius

        # ダイナミクス(この例ではEnvと全く同じ正確なダイナミクスが利用可能とする)
        self._A = np.zeros((self.X_DIM, self.X_DIM))
        self._A[0:6, 3:9] = np.identity(6)
        self._A[6, 3] = self.ALPHA_1X
        self._A[7, 4] = self.ALPHA_1Y
        self._A[8, 5] = self.ALPHA_1Z
        self._A[6, 6] = self.ALPHA_2X
        self._A[7, 7] = self.ALPHA_2Y
        self._A[8, 8] = self.ALPHA_2Z
        self._B = np.zeros((self.X_DIM, self.U_DIM))
        self._B[6, 0] = self.BETA_X
        self._B[7, 1] = self.BETA_Y
        self._B[8, 2] = self.BETA_Z

        # 速度にのみ重みをかける
        self._Q = np.zeros((self.X_DIM, self.X_DIM))
        self._Q[3, 3] = vel_weights[0]
        self._Q[4, 4] = vel_weights[1]
        self._Q[5, 5] = vel_weights[2]
        self._R = np.diag([input_weight] * self.U_DIM)

    def x_dot(self, x: NDArray, u: NDArray):
        assert x.shape == (self.X_DIM,)
        assert u.shape == (self.U_DIM,)

        xd = self._A @ x + self._B @ u
        return xd

    def gradient_terminal_cost_state(self, t: float, x: NDArray, x_des: NDArray) -> NDArray:
        assert x.shape == (self.X_DIM,)
        assert x_des.shape == (self.X_DIM,)

        # この例では特に終端コストを考えていないため，gradient_hamiltonian_stateの結果とほぼ同じになる
        pBpx = np.zeros((self.X_DIM,))
        denom = (self._pole_radius**2 - ((x[0] - self._pole_x)**2 + (x[1] - self._pole_y)**2))**3
        pBpx[0] = 4. * (x[0] - self._pole_x) / denom
        pBpx[1] = 4. * (x[1] - self._pole_y) / denom

        res = self._Q @ (x - x_des) + (1. / self._rx) * pBpx
        return res

    def gradient_hamiltonian_state(
        self,
        t: float,
        x: NDArray,
        x_des: NDArray,
        u: NDArray,
        lam: NDArray,
        rho: NDArray,
    ) -> NDArray:
        assert x.shape == (self.X_DIM,), x.shape
        assert x_des.shape == (self.X_DIM,), x_des.shape
        assert lam.shape == (self.X_DIM,), lam.shape

        pBpx = np.zeros((self.X_DIM,))
        denom = (self._pole_radius**2 - ((x[0] - self._pole_x)**2 + (x[1] - self._pole_y)**2))**3
        pBpx[0] = 4. * (x[0] - self._pole_x) / denom
        pBpx[1] = 4. * (x[1] - self._pole_y) / denom

        pHpx = self._Q @ (x - x_des) + (1. / self._rx) * pBpx + self._A.T @ lam
        return pHpx

    def gradient_hamiltonian_input(
        self,
        t: float,
        x: NDArray,
        x_des: NDArray,
        u: NDArray,
        lam: NDArray,
        rho: NDArray,
    ) -> NDArray:
        assert u.shape == (self.U_DIM,), u.shape
        assert lam.shape == (self.X_DIM,), lam.shape

        pPpu = (4. * max(0., u[0]**2 + u[1]**2 - self.MAX_ANGLE**2)) * np.array([u[0], u[1], 0.])

        pHpu = self._R @ u + self._ru * pPpu + self._B.T @ lam
        return pHpu

    def constraint(self, t: float, x: NDArray, x_des: NDArray, u: NDArray) -> NDArray:
        return np.zeros((self.C_DIM,))


制御入力の範囲や状態方程式内の各値については書籍に記載されていないため，適当に設定しています．\
また，等式制約は存在しないため，`constraint()`は長さ0の空のベクトルを返すようにしています．

次に，状態の誤差にかかる重み$\bm{Q}$と予測区間内の目標状態$\bm{x}_{\mathrm{des}}$の与え方について考えます．\
先に実装を出しましたが，この例では9次元の状態のうち速度にのみ重みをかけるようにしています．本来の目的はドローンを目標位置に移動させることですが，位置に直接重みをかけないのには理由があります．位置の2乗誤差をコストとするとドローンと目標位置との距離によってコストのスケールが大きく異なるため，一般に有効なハイパーパラメータを見つけるのが困難になると考えられるからです．一般に，MPCに与える目標状態は予測区間(リアルタイム制御なら1secくらいのオーダ)に到達可能だと考えられる値から大きく外れるべきではないでしょう．

よって，ドローンを目標位置に導くための目標速度について考える必要があります．ややヒューリスティックですが，この例では一定値と2次関数を組み合わせて以下のように目標速度を設定します．
$$ \bm{v}_{\mathrm{des}} = v_{\mathrm{des}} \frac{\bm{p}_{\mathrm{diff}}}{d} $$
$$
v_{\mathrm{des}} =
\begin{cases}
v_{\mathrm{max}} \ & (d > d_{\mathrm{th}}) \\
v_{\mathrm{max}} \ (1 - (1 - \frac{d}{d_{\mathrm{th}}})^n) & (d \le d_{\mathrm{th}})
\end{cases}
$$
$$ \bm{p}_{\mathrm{diff}} = \bm{p}_{\mathrm{goal}} - \bm{p}_{\mathrm{cur}}, \ d = ||\bm{p}_{\mathrm{diff}}|| $$
$d_{\mathrm{th}}$は最大速度から曲線に入る際の目標位置と現在の位置との距離に関する閾値であり，$n > 0$が大きいほど$d = 0$付近で急激に目標速度が低下します．\
試しに$n = 2$の場合の距離と目標速度の関係をプロットすると次のようになります．

In [ ]:
from matplotlib import pyplot as plt

def speed_profile(v_max, d, d_th, n):
    return np.where(d > d_th, v_max, v_max * (1. - (1. - d / d_th)**n))

v_max = 0.5
d = np.linspace(0., 3., 1000)
d_th = 1.
n = 2.

v = speed_profile(v_max, d, d_th, n)

plt.figure(figsize=(12, 9))
plt.plot(d, v)
plt.xlabel(r'The distance between $p_{\mathrm{cur}}$ and $p_{\mathrm{goal}}$')
plt.ylabel(r'$v_{\mathrm{des}}$')
plt.show()

$\bm{p}_{\mathrm{cur}}$と$\bm{p}_{\mathrm{goal}}$の距離が小さくなるにつれて目標速度も小さくなっていることがわかります．

それでは最終目標位置と現在の位置から目標速度を算出するプランナを実装します．基底クラス`BasePlanner`の子クラスとして定義し，仮想関数をオーバーライドすればよいです．

In [ ]:
from numpy import linalg as LA

from nmpc_cgmres import BasePlanner


class DroneStaticObstaclePlanner(BasePlanner):

    X_DIM = 9
    U_DIM = 3
    EPS = 1e-9

    def __init__(self, pred_len: int, max_speed: float, dist_threshold: float):
        assert max_speed > 0.
        assert dist_threshold > 0.

        super().__init__(pred_len=pred_len)

        self._max_speed = max_speed
        self._dist_threshold = dist_threshold

    def plan(self, x: NDArray, x_goal: NDArray) -> NDArray:
        assert x.shape == (self.X_DIM,), x.shape
        assert x_goal.shape == (self.X_DIM,), x_goal.shape

        pos_cur = x[0:3]
        pos_goal = x_goal[0:3]

        dist = LA.norm(pos_cur - pos_goal)
        if dist > self._dist_threshold:
            speed = self._max_speed
        else:
            ratio = dist / self._dist_threshold
            speed = ratio * (2. - ratio) * self._max_speed

        # 1ステップごとの目標状態
        vel_des = (pos_goal - pos_cur) * (speed / (dist + self.EPS))
        x_des = np.zeros((self.X_DIM,))
        x_des[3:6] = vel_des

        # 予測区間全体の目標状態．予測区間に渡って一定とする．
        X_des = np.tile(x_des, (self._pred_len, 1))

        return X_des


先程の例と同様に$n = 2$とし，少し式変形しています．\
また，簡単のため目標状態は予測区間に渡って一定としています．よりロバスト性を高めるためには目標状態の軌道を現在の状態から指数関数的に目標状態に近づけるなどの工夫が考えられます．詳しくは[モデル予測制御](https://www.tdupress.jp/book/b349347.html)などをご参照ください．

コントローラ側としてはこれで完成です．あとはドローンから現在の状態を受け取り，`DroneStaticObstaclePlanner`で目標状態を作成し，`DroneStaticObstacleModel`を読み込ませた`NmpcCgmres`が出した最適制御入力をドローンに入力すればよいのですが，肝心のドローンが手元に無いためシミュレーション環境も自作する必要があります．\
基底クラス`BaseEnv`を用いて描画機能付きの簡単な制御プラントを作ることができます．先程までと同様に継承して仮想関数をオーバーライドします．

In [ ]:
from typing import Tuple
from matplotlib.axes import Axes

from nmpc_cgmres import BaseEnv
from nmpc_cgmres.util import circle, runge_kutta_4


class DroneStaticObstacleEnv(BaseEnv):

    X_DIM = 9
    U_DIM = 3
    MAX_ANGLE = np.deg2rad(30.)
    MAX_THRUST = 10.

    ALPHA_1X = 0.
    ALPHA_1Y = 0.
    ALPHA_1Z = 0.
    ALPHA_2X = 0.
    ALPHA_2Y = 0.
    ALPHA_2Z = 0.
    BETA_X = 1.
    BETA_Y = 1.
    BETA_Z = 1.

    def __init__(
        self,
        start_pos: NDArray,
        goal_pos: NDArray,
        pole_pos: NDArray,
        pole_radius: float,
        dt: float,
        task_horizon: float,
    ):
        assert start_pos.shape == (3,)
        assert goal_pos.shape == (3,)
        assert pole_pos.shape == (2,)
        assert pole_radius > 0.
        assert dt > 0.
        assert task_horizon > 0

        super().__init__()

        self._start_pos = start_pos
        self._goal_pos = goal_pos
        self._pole_pos = pole_pos
        self._pole_radius = pole_radius
        self._dt = dt
        self._max_step = int(task_horizon / dt)
        self._step_count = 0
        self._x = np.empty((self.X_DIM,))
        self._x_history = np.empty((self._max_step + 1, self.X_DIM))

        # 制御入力の制限
        self._u_lb = np.array([-self.MAX_ANGLE, -self.MAX_ANGLE, -self.MAX_THRUST])
        self._u_ub = np.array([self.MAX_ANGLE, self.MAX_ANGLE, self.MAX_THRUST])

        # ダイナミクス
        self._A = np.zeros((self.X_DIM, self.X_DIM))
        self._A[0:6, 3:9] = np.identity(6)
        self._A[6, 3] = self.ALPHA_1X
        self._A[7, 4] = self.ALPHA_1Y
        self._A[8, 5] = self.ALPHA_1Z
        self._A[6, 6] = self.ALPHA_2X
        self._A[7, 7] = self.ALPHA_2Y
        self._A[8, 8] = self.ALPHA_2Z
        self._B = np.zeros((self.X_DIM, self.U_DIM))
        self._B[6, 0] = self.BETA_X
        self._B[7, 1] = self.BETA_Y
        self._B[8, 2] = self.BETA_Z

    def reset(self) -> NDArray:
        self._step_count = 0
        self._x = np.r_[self._start_pos, np.zeros((6,))]
        self._x_history = np.empty((self._max_step + 1, self.X_DIM))
        self._x_history[0, :] = self._x[:]

        return self._x.copy()

    def step(self, u: NDArray) -> Tuple[NDArray, float, bool, dict]:
        assert u.shape == (self.U_DIM,)

        # dynamics
        u = np.clip(u, self._u_lb, self._u_ub)
        self._x = runge_kutta_4(self._x, u, self._x_dot, self._dt)

        # cost
        cost = None  # 特に使わないためグローバルコストは定めない

        # done
        self._step_count += 1
        done = self._step_count >= self._max_step

        # info
        info = dict()

        # 記録
        self._x_history[self._step_count, :] = self._x[:]

        return self._x.copy(), cost, done, info

    def _x_dot(self, x: NDArray, u: NDArray) -> NDArray:
        return self._A @ x + self._B @ u

    def _set_imgs(self, axis: Axes):
        self._imgs.clear()
        self._imgs['drone'] = axis.plot([], [], marker='o', c='r', markersize=10)[0]
        self._imgs['traj'] = axis.plot([], [], c='k', linestyle='dashed')[0]
        self._imgs['start'] = axis.plot([], [], marker='o', c='k', markersize=10)[0]
        self._imgs['goal'] = axis.plot([], [], marker='o', c='k', markersize=10)[0]
        self._imgs['pole'] = axis.plot([], [], marker='o', c='k', markersize=10)[0]
        self._imgs['barrier'] = axis.plot([], [], c='k', linestyle='dashed')[0]

        margin = 0.5
        x_min = min(self._start_pos[0], self._goal_pos[0], self._pole_pos[0] - self._pole_radius)
        x_max = max(self._start_pos[0], self._goal_pos[0], self._pole_pos[0] + self._pole_radius)
        y_min = min(self._start_pos[1], self._goal_pos[1], self._pole_pos[1] - self._pole_radius)
        y_max = max(self._start_pos[1], self._goal_pos[1], self._pole_pos[1] + self._pole_radius)
        axis.set_xlim([x_min - margin, x_max + margin])
        axis.set_ylim([y_min - margin, y_max + margin])

    def _plot_func(self, i: int):
        self._imgs['drone'].set_data(self._x_history[i, 0], self._x_history[i, 1])
        self._imgs['traj'].set_data(self._x_history[:i, 0], self._x_history[:i, 1])
        self._imgs['start'].set_data(self._start_pos[0], self._start_pos[1])
        self._imgs['goal'].set_data(self._goal_pos[0], self._goal_pos[1])
        self._imgs['pole'].set_data(self._pole_pos[0], self._pole_pos[1])
        self._imgs['barrier'].set_data(
            *circle(self._pole_pos[0], self._pole_pos[1], self._pole_radius)
        )

    @property
    def max_step(self) -> int:
        return self._max_step

    @property
    def n_frames(self) -> int:
        return len(self._x_history)


`DroneStaticObstacleEnv`では`DroneStaticObstacleModel`で定義したものと全く同じダイナミクスで状態遷移を行います．つまり，この例ではモデル化誤差は0となります．これは実際の制御では考えられませんが，チュートリアルを進める上では問題ないのであまり気にしないことにします．\
最も重要なメソッドは`step()`です．コントローラから受け取った制御入力を用いて環境を1ステップ進め，次の状態，コスト，終了フラグ，その他の情報を返します．この例では4次のルンゲクッタ法で積分を行い，終了条件は先に定めたエピソード長にのみ依存するとしています．\
また，後の描画のために，状態の軌跡を保存する`_x_history`をインスタンス変数として持ち，初期設定を行う`_set_imgs()`と描画を行う`_plot_func()`もオーバーライドしています．

それでは実際にシミュレーションし，結果を描画してみます．

In [ ]:
from nmpc_cgmres import NmpcCgmres


def run(
    env: DroneStaticObstacleEnv,
    planner: DroneStaticObstaclePlanner,
    controller: NmpcCgmres,
    x_goal: NDArray,
    dt: float,
) -> Tuple[NDArray, NDArray]:
    done = False
    x = env.reset()
    history_x, history_u = [], []
    step_count = 0

    while not done:
        # Plan
        x_des = planner.plan(x, x_goal)

        # Obtain sol
        u = controller.step(x, x_des, dt)

        # Step
        next_x, cost, done, info = env.step(u)

        # Save
        history_x.append(x.copy())
        history_u.append(u.copy())

        # Update
        x = next_x
        step_count += 1

        # Display
        print('\r' + f'Step: {step_count}/{env.max_step}', end='')

    print()

    return np.array(history_x), np.array(history_u)


# パラメータ
start_pos = [-1.4, -0.4, 0.]
goal_pos = [2., 0.5, 1.]
pole_pos = [0.5, 0.]
pole_radius = 0.4
vel_weights = [1., 1., 1.]
input_weight = 1e-2
rx = 100.
ru = 1.
controller_freq = 100.
task_horizon = 20.
pred_len = 10
max_speed = 0.5
dist_threshold = 1.
pred_horizon = 0.5
horizon_half_time = 0.3
zeta_coef = 1.

dt = 1. / controller_freq  # 制御周期

# プラント，モデル，プランナ，コントローラを作成
env = DroneStaticObstacleEnv(
    start_pos=np.array(start_pos),
    goal_pos=np.array(goal_pos),
    pole_pos=np.array(pole_pos),
    pole_radius=pole_radius,
    dt=dt,
    task_horizon=task_horizon,
)
model = DroneStaticObstacleModel(
    pole_pos=np.array(pole_pos),
    pole_radius=pole_radius,
    vel_weights=np.array(vel_weights),
    input_weight=input_weight,
    rx=rx,
    ru=ru,
)
planner = DroneStaticObstaclePlanner(
    pred_len=pred_len,
    max_speed=max_speed,
    dist_threshold=dist_threshold,
)
controller = NmpcCgmres(
    model=model,
    pred_horizon=pred_horizon,
    pred_len=pred_len,
    horizon_half_time=horizon_half_time,
    zeta_coef=zeta_coef,
)

# シミュレーション
x_goal = np.array(goal_pos + [0.] * 6)
history_x, history_u = run(env, planner, controller, x_goal, dt)

# 結果を描画
t = np.linspace(0., task_horizon, env.n_frames - 1)

plt.figure(figsize=(8, 6))
plt.plot(t, history_x[:, 0], label='x')
plt.plot(t, history_x[:, 1], label='y')
plt.plot(t, history_x[:, 2], label='z')
plt.xlabel('Time [sec]')
plt.ylabel('Position [m]')
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(t, history_u[:, 0], label='u0')
plt.plot(t, history_u[:, 1], label='u1')
plt.plot(t, history_u[:, 2], label='u2')
plt.xlabel('Time [sec]')
plt.ylabel('System Input')
plt.legend()
plt.show()


正しく目標位置`goal_pos`で停止していることがわかります．\
実際に障害物を避けて目標地点に到達していることを確かめるために，最後に動画を出力してみます．

In [ ]:
%matplotlib widget

ani = env.get_animation(dt=dt, n_frames=env.n_frames)
plt.show()

赤丸で示されたドローンがきれいに障害物を避けてゴールに到達する様子が再生されると思います．\
パラメータを変えて色々試してみると面白いです．

以上